<a href="https://colab.research.google.com/github/9105507/UFV_VISUALIZACION_DE_DATOS/blob/main/Ejercicios_Clase/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctica de Python (ventas en Europa)**

Preguntas que hay que responder:

1. Como se distribuyen las ventas realizadas en:
  * Cada país.
  * Cada mes y año.
  * Cada marca.

2. Cual es la tendencia y estacionalidad de:
  * Todas las ventas del país con menos venta.
  * La marca con más ventas.

3. Cuales son las predicciones hechas en España y como de
buenas son.

Se importan librerías usadas.

In [42]:
import pandas as pd
import numpy as np
import plotly.express as px

Se cargan los datos.

In [43]:
data = pd.read_csv('datos_ejercicio_ventas.csv', delimiter=';')
data.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,"7,54356E+15"
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,"5,60031E+15"
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,"8,8502E+15"
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,"3,63225E+15"
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,"3,96176E+16"


Tamaño de los datos.

In [44]:
data.shape

(18666, 8)

Info de los datos.

In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18666 entries, 0 to 18665
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   COUNTRY        18666 non-null  object 
 1   SUBBRAND       18666 non-null  object 
 2   YEAR           18666 non-null  int64  
 3   MONTH          18666 non-null  int64  
 4   SCENARIO       18666 non-null  object 
 5   FORECAST       17766 non-null  object 
 6   FORECAST_YEAR  17766 non-null  float64
 7   AMOUNT         18666 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 1.1+ MB


DATASET:
CADA REGISTRO REPRESENTA LA CANTIDAD VENDIDA DE UN PRODUCTO EN UN PAÍS Y EN UN INSTANTE TEMPORAL (DADO EN MESES). ESTA CANTIDAD PUEDE SER UN DATO REAL O UNA PREDICCIÓN HECHA.
* 'COUNTRY': STRING, PAÍS, VARIABLE CATEGÓRICA.
* 'SUBBRAND': STRING, PRODUCTO, VARIABLE CATEGÓRICA.
* 'YEAR': INT, AÑO DEL VALOR, VARIABLE DISCRETA.
* 'MONTH': INT, MES DEL VALOR, VARIABLE DISCRETA.
* 'SCENARIO': STRING, INDICA SI EL VALOR ES REAL O UNA PREDICCIÓN, VARIABLE CATEGÓRICA.
* 'FORECAST': STRING, SI ES UNA PREDICCIÓN, INDICA EL PERÍODO DESDE EL QUE SE HACE LA PREDICCIÓN. MÁS CONCRETAMENTE, 'P0X' indica que se está en el final del mes X-1. LO INDICA A MES VENCIDO. 'P02' ES FINALES DE ENERO / PRINCIPIOS DE FEBRERO. VARIABLE CATEGORICA
* 'FORECAST_YEAR': FLOAT, EL AÑO DESDE EL QUE SE ESTÁ HACIENDO LA PREVISIÓN, VARIABLE DISCRETA.
* 'AMOUNT': STRING, AUNQUE REPRESENTA UN NÚMERO. E+ALGO SE REFIERE A E A LA ALGO. REPRESENTA UNA VARIABLE CONTINUA.

**NULLS EN 'FORECAST' Y 'FORECAST_YEAR', ADEMÁS EL MISMO Nº DE NULLS EN CADA UNA**

Nº de Nan's por variable:

In [46]:
data.isna().sum()

,0
COUNTRY,0
SUBBRAND,0
YEAR,0
MONTH,0
SCENARIO,0
FORECAST,900
FORECAST_YEAR,900
AMOUNT,0


In [47]:
data[(data['FORECAST'].isna()) & (data['FORECAST_YEAR'].isna())]

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,"1,88595E+16"
278,Portugal,7up (L3),2023,3,actual,NaN,NaN,"2,93497E+15"
279,Portugal,7up (L3),2023,10,actual,NaN,NaN,"3,48447E+16"
280,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,"1,17255E+16"
281,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,"3,78486E+15"
...,...,...,...,...,...,...,...,...
18595,Great Britain,Lipton (L3),2023,9,actual,NaN,NaN,"8,49263E+15"
18596,Italy,Pepsi Regular (L3),2023,9,actual,NaN,NaN,"1,56257E+16"
18597,Great Britain,7up Free (L3),2023,9,actual,NaN,NaN,"1,29323E+15"
18598,Great Britain,Mountain Dew (L3),2023,9,actual,NaN,NaN,"2,06646E+16"


**TODOS LOS NAN'S DE 'FORECAST' SON TAMBIÉN TODOS LOS NAN'S DE 'FORECAST_YEAR'**

In [48]:
data[data['SCENARIO'] == 'actual']

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
277,Portugal,Pepsi Max (L3),2023,10,actual,NaN,NaN,"1,88595E+16"
278,Portugal,7up (L3),2023,3,actual,NaN,NaN,"2,93497E+15"
279,Portugal,7up (L3),2023,10,actual,NaN,NaN,"3,48447E+16"
280,Great Britain,7up Free (L3),2023,10,actual,NaN,NaN,"1,17255E+16"
281,Norway,Pepsi Regular (L3),2023,10,actual,NaN,NaN,"3,78486E+15"
...,...,...,...,...,...,...,...,...
18595,Great Britain,Lipton (L3),2023,9,actual,NaN,NaN,"8,49263E+15"
18596,Italy,Pepsi Regular (L3),2023,9,actual,NaN,NaN,"1,56257E+16"
18597,Great Britain,7up Free (L3),2023,9,actual,NaN,NaN,"1,29323E+15"
18598,Great Britain,Mountain Dew (L3),2023,9,actual,NaN,NaN,"2,06646E+16"


**TODOS LOS NAN'S SE DAN CUANDO EL DATO ES REAL. ESTO TIENE SENTIDO, YA QUE SI TENGO EL DATO REAL, NO HAY PREDICCIÓN**

Se juntan 'MONTH' y 'YEAR' en una sola variable.

In [49]:
data['DATE'] = pd.to_datetime(data[['YEAR', 'MONTH']].assign(DAY=1))

**COSAS INTERESANTES QUE ANALIZAR**:
1. Nº ACTUALS Y FORECAST.
2. HORIZONTE DE PREVISIÓN. PISTA: ES EL MISMO SIEMPRE.
3. Nº PAÍSES, PRODUCTOS.
4. HISTÓRICO DE DATOS DE ACTUALS / FORECAST.
5. Nº DE FORECAST DISTINTOS.

**Nº ACTUALS Y FORECAST.**

In [50]:
data[data['SCENARIO'] == 'actual'].shape[0]

900

HAY 900 DATOS REALES.

In [51]:
data[data['SCENARIO'] == 'AI_forecast'].shape[0]

17766

HAY 17766 VALORES PREDICHOS.

**HORIZONTE DE PREVISIÓN.**

El instante en el que se predice para una marca de un país viene dado por la combinación de 'FORECAST' y 'FORECAST_YEAR'. Para ver cuál es el horizonte de previsión para cada predicción hecha, basta con ver cuántas combinaciones únicas de 'MONTH' y 'YEAR' hay para cada combinación de ('FORECAST', 'FORECAST_YEAR', 'COUNTRY', 'SUBBRAND'). Así, Ss calculan los horizontes de previsión únicos en el dataset.

In [52]:
df_grouped = data.groupby(['FORECAST', 'FORECAST_YEAR', 'COUNTRY', 'SUBBRAND'])

unique_dates_count = df_grouped['DATE'].nunique().reset_index(name='UNIQUE_DATES_COUNT')
unique_dates_count.head(10)

,FORECAST,FORECAST_YEAR,COUNTRY,SUBBRAND,UNIQUE_DATES_COUNT
0,AI_P02F,2023.0,Czech,7up (L3),18
1,AI_P02F,2023.0,Czech,Lipton (L3),18
2,AI_P02F,2023.0,Czech,Mountain Dew (L3),18
3,AI_P02F,2023.0,Czech,Pepsi Max (L3),18
4,AI_P02F,2023.0,Czech,Pepsi Regular (L3),18
5,AI_P02F,2023.0,Denmark,7up (L3),18
6,AI_P02F,2023.0,Denmark,7up Free (L3),18
7,AI_P02F,2023.0,Denmark,Mountain Dew (L3),18
8,AI_P02F,2023.0,Denmark,Pepsi Max (L3),18
9,AI_P02F,2023.0,Denmark,Pepsi Regular (L3),18


In [53]:
unique_dates_count['UNIQUE_DATES_COUNT'].unique().tolist()

[18]

El horizonte de previsión es el mismo para todas las predicciones realizadas, siempre es 18.

**Nº PAÍSES, PRODUCTOS.**

In [54]:
len(data['COUNTRY'].unique().tolist())

9

In [55]:
data['COUNTRY'].unique().tolist()

['Portugal',
 'Great Britain',
 'Spain',
 'Hungary',
 'Norway',
 'Denmark',
 'Netherlands',
 'Italy',
 'Czech']

In [56]:
len(data['SUBBRAND'].unique().tolist())

6

In [57]:
data['SUBBRAND'].unique().tolist()

['Lipton (L3)',
 'Pepsi Max (L3)',
 '7up (L3)',
 'Pepsi Regular (L3)',
 'Mountain Dew (L3)',
 '7up Free (L3)']

**HISTÓRICO DE DATOS DE ACTUALS / FORECAST.**

In [58]:
df = data[data['SCENARIO'] == 'actual'].groupby(['COUNTRY', 'SUBBRAND'])
# df.indices
group = df.get_group(('Czech', '7up (L3)'))  # Devuelve las filas correspondientes a este grupo específico
group['AMOUNT'].iloc[2]

'8,10474E+15'

AMOUNT SON STRINGS, HAY QUE CONVERTIR A INTS

In [59]:
# Función para convertir los valores a enteros
def convert_to_int(value):
    try:
        # Reemplaza la coma decimal y convierte el valor a float, luego a entero
        return int(float(value.replace(',', '.')))
    except ValueError:
        # Si hay un error de conversión, devuelve NaN o un valor por defecto
        return None

# Aplica la función a la columna 'AMOUNT'
data['AMOUNT'] = data['AMOUNT'].apply(convert_to_int)
df = data[data['SCENARIO'] == 'actual'].groupby(['COUNTRY', 'SUBBRAND'])
# df.indices
group = df.get_group(('Czech', '7up (L3)'))  # Devuelve las filas correspondientes a este grupo específico
group['AMOUNT'].iloc[2]


8104740000000000

Visualización de la serie temporal real de alguna de las combinaciones ('COUNTRY', 'SUBBRAND'):

In [60]:
# Especificar el producto y país
producto = '7up (L3)'
país = 'Czech'

# Filtrar y agrupar los datos por escenario 'actual', 'COUNTRY' y 'SUBBRAND'
df = data[data['SCENARIO'] == 'actual'].groupby(['COUNTRY', 'SUBBRAND'])

# Obtener un grupo específico ('Czech', '7up (L3)') y hacer una copia explícita
group = df.get_group((país, producto)).copy()

# Ordenar el DataFrame por la columna 'DATE'
group = group.sort_values(by='DATE')

# Visualizar la serie temporal con Plotly Express, incluyendo marcadores
fig = px.line(group, x='DATE', y='AMOUNT',
              title=f'ACTUALS para {país} - {producto}',
              labels={'DATE': 'Date', 'AMOUNT': 'Amount'},
              markers=True)  # Añadir marcadores a los puntos

# Mostrar la gráfica interactiva
fig.show()

Visualización de la serie temporal de las predicciones hechas desde un 'forecast' de un año para alguna de las combinaciones ('COUNTRY', 'SUBBRAND'):

In [61]:
# Especificar el producto y país
producto = '7up (L3)'
país = 'Czech'
forecast = 'AI_P04F'
year = 2023

# Filtrar y agrupar los datos por escenario 'AI_forecast', 'COUNTRY', 'SUBBRAND', 'FORECAST' y 'FORECAST_YEAR'
df = data[data['SCENARIO'] == 'AI_forecast'].groupby(['COUNTRY', 'SUBBRAND', 'FORECAST', 'FORECAST_YEAR'])

# Obtener un grupo específico ('Czech', '7up (L3)', 'AI_P04F', 2023) y hacer una copia explícita
group = df.get_group((país, producto, forecast, year)).copy()

# Ordenar el DataFrame por la columna 'DATE'
group = group.sort_values(by='DATE')

# Visualizar la serie temporal con Plotly Express, incluyendo marcadores
fig = px.line(group, x='DATE', y='AMOUNT',
              title=f'PREDICCIONES para {país} - {producto} desde {forecast} - {year}',
              labels={'DATE': 'Date', 'AMOUNT': 'Amount'},
              markers=True)  # Añadir marcadores a los puntos

# Mostrar la gráfica interactiva
fig.show()

**Nº DE FORECAST DISTINTOS.**

¿Cuántas predicciones distintas hay en la base de datos?

no se a que se refiere.

## 1. Como se distribuyen las ventas realizadas en:

### Cada país.

### Cada mes y año.

### Cada marca.